<a href="https://colab.research.google.com/github/ParsaMohammadpour/EIGByz/blob/main/Class-Assignments/4th-week/collaborative_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS246 - Colab 4
## Collaborative Filtering

In [ ]:
from IPython.display import Image
print("Colab 4 Mascot")
Image(url='https://media4.giphy.com/media/wtdVYmaRWJ1PyPcc8e/giphy.gif?cid=ecf05e478jdbzo92b89f2b5ud5184xda9cen1xce4bwmjeyu&ep=v1_stickers_search&rid=giphy.gif&ct=s',width=150)

Colab 4 Mascot


### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive2
!apt install openjdk-8-jdk-headless -qq
!pip install findspark
!pip install pyngrok
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 34 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 126333 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u442-b06~us1-0ubuntu1~22.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u442-b06~us1

Now we authenticate a Google Drive client to download the files we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1QtPy_HuIMSzhtYllT3-WeM3Sqg55wK_D'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.training')

id='1ePqnsQTJRRvQcBoF2EhoPU8CU1i5byHK'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.test')

id='1ncUBWdI5AIt3FDUJokbMqpHD2knd5ebp'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.item')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)

In [ ]:
import findspark

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName('Colab_4') \
        .getOrCreate()

In [ ]:
from pyngrok import ngrok, conf
import getpass

print("Enter your authtoken, which can be copied "
"from https://dashboard.ngrok.com/get-started/your-auth-token")
conf.get_default().auth_token = getpass.getpass()

ui_port = 4050
public_url = ngrok.connect(ui_port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{ui_port}\"")

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/2wJ6m7HSixFhoikiOTtm0tNFZVx_i2UgW52nQqxvog3CxkRe
··········
 * ngrok tunnel "https://91b9-34-23-185-9.ngrok-free.app" -> "http://127.0.0.1:4050"


### Data Loading

In this Colab, we will be using the [MovieLens dataset](https://grouplens.org/datasets/movielens/), specifically the 100K dataset (which contains in total 100,000 ratings from 1000 users on ~1700 movies).

We load the ratings data in a 80%-20% ```training```/```test``` split, while the ```items``` dataframe contains the movie titles associated to the item identifiers.

In [ ]:
schema_ratings = StructType([
    StructField("user_id", IntegerType(), False),
    StructField("item_id", IntegerType(), False),
    StructField("rating", IntegerType(), False),
    StructField("timestamp", IntegerType(), False)])

schema_items = StructType([
    StructField("item_id", IntegerType(), False),
    StructField("movie", StringType(), False)])

training = spark.read.option("sep", "\t").csv("MovieLens.training", header=False, schema=schema_ratings)
test = spark.read.option("sep", "\t").csv("MovieLens.test", header=False, schema=schema_ratings)
items = spark.read.option("sep", "|").csv("MovieLens.item", header=False, schema=schema_items)

In [ ]:
training.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [ ]:
items.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- movie: string (nullable = true)



### Your task

Let's compute some stats!  What is the number of ratings in the training and test dataset? How many movies are in our dataset?

In [ ]:
''' 3 lines of code in total expected.
For sub-parts of the question (if any), creating different cells of code would be recommended.'''
# YOUR CODE HERE
training_count = training.count()
print(f'training count: {training_count}')

training count: 80000


Using the training set, train a model with the Alternating Least Squares method available in the Spark MLlib: [https://spark.apache.org/docs/latest/ml-collaborative-filtering.html](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html)

In [ ]:
''' 5-8 lines of code in total expected but can differ based on your style.
For sub-parts of the question (if any), creating different cells of code would be recommended.'''
# YOUR CODE HERE
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [ ]:
training.dtypes

[('user_id', 'int'),
 ('item_id', 'int'),
 ('rating', 'int'),
 ('timestamp', 'int')]

In [ ]:
als = ALS(maxIter=20, regParam=0.01, userCol="user_id",
          itemCol="item_id", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

Now compute the RMSE on the test dataset. **Please refer to Gradescope for the hyperparameters**.


In [ ]:
''' 4-5 lines of code in total expected but can differ based on your style.
For sub-parts of the question (if any), creating different cells of code would be recommended.'''
# YOUR CODE HERE
test.dtypes

[('user_id', 'int'),
 ('item_id', 'int'),
 ('rating', 'int'),
 ('timestamp', 'int')]

In [ ]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f'rmse error = {rmse}')

rmse error = 1.1680825116162274


At this point, you can use the trained model to produce the top-K recommendations for each user.

In [ ]:
''' 9-10 lines of code in total expected but can differ based on your style.
For sub-parts of the question (if any), creating different cells of code would be recommended.'''
# YOUR CODE HERE

recomm_no = 10
user_recommendation = model.recommendForAllUsers(recomm_no)
user_recommendation.show(n=10)

+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|      1|[{1470, 6.906544}...|
|      3|[{1093, 7.289514}...|
|      5|[{1286, 8.956291}...|
|      6|[{1066, 7.3400917...|
|      9|[{1066, 10.128086...|
|     12|[{277, 7.69035}, ...|
|     13|[{982, 7.0798106}...|
|     15|[{1311, 8.188713}...|
|     16|[{1131, 8.130413}...|
|     17|[{580, 8.816478},...|
+-------+--------------------+
only showing top 10 rows



In [25]:
recomm_no = 10
movie_recommendation = model.recommendForAllItems(recomm_no)
movie_recommendation.show(n=10)

+-------+--------------------+
|item_id|     recommendations|
+-------+--------------------+
|      1|[{688, 5.7401204}...|
|      3|[{418, 8.319965},...|
|      5|[{39, 7.8715954},...|
|      6|[{471, 12.761428}...|
|      9|[{362, 9.086249},...|
|     12|[{415, 6.6650763}...|
|     13|[{122, 7.883937},...|
|     15|[{364, 6.154531},...|
|     16|[{762, 8.316993},...|
|     17|[{418, 7.1645365}...|
+-------+--------------------+
only showing top 10 rows



Once you have working code for each cell above, **head over to Gradescope, carefully read the questions, and submit your solution for this Colab**! As a sanity check, the 4th most recommended movie under the setting of Q2.6 should be `Star Wars (1977)`.